This analysis has been done on data version 440 (10/17/2021). Please extract the content inside **./data**

### Below, you can set preferences on the region to analyze and line display limit

In [9]:
# "Data is included for the IN, US, GB, DE, CA, FR, RU, BR, MX, KR, and JP regions 
# (India, USA, Great Britain, Germany, Canada, France, Russia, Brazil, Mexico, South Korea, and, Japan respectively)"
# please enter region in two letter format (like first line) and in quotation on variable region
region = "US"

display_limit = 1000

For the initial analysis, we will only be looking at data from the US.

In [10]:
import json
import pandas as pds
import re

The below code checks for data existance as well as some initializations

In [44]:
from os import listdir

data_files_location = {}
categories = {}

for f in listdir('./data'):
    if f.endswith('.json'):
        temp = json.load(open(f'./data/{f}'))
        for i in temp['items']:
            categories[i['id']] = i['snippet']['title']
    if f.endswith('.csv'):
        data_files_location[f[:2]] = f'./data/{f}'

if not categories:
    raise FileNotFoundError('can not locate any json category files')
    
if not data_files_location:
    raise FileNotFoundError('no CSV data files are located')


Loading and assiging CSV files onto variables with Pandas

In [12]:
data = pds.read_csv(data_files_location[region])
data

,video_id,title,publishedAt,channelId,channelTitle,categoryId,trending_date,tags,view_count,likes,dislikes,comment_count,thumbnail_link,comments_disabled,ratings_disabled,description
0,3C66w5Z0ixs,I ASKED HER TO BE MY GIRLFRIEND...,2020-08-11T19:20:14Z,UCvtRTOMP2TqYqu51xNrqAzg,Brawadis,22,2020-08-12T00:00:00Z,brawadis|prank|basketball|skits|ghost|funny vi...,1514614,156908,5855,35313,https://i.ytimg.com/vi/3C66w5Z0ixs/default.jpg,False,False,SUBSCRIBE to BRAWADIS ▶ http://bit.ly/Subscrib...
1,M9Pmf9AB4Mo,Apex Legends | Stories from the Outlands – “Th...,2020-08-11T17:00:10Z,UC0ZV6M2THA81QT9hrVWJG3A,Apex Legends,20,2020-08-12T00:00:00Z,Apex Legends|Apex Legends characters|new Apex ...,2381688,146739,2794,16549,https://i.ytimg.com/vi/M9Pmf9AB4Mo/default.jpg,False,False,"While running her own modding shop, Ramya Pare..."
2,J78aPJ3VyNs,I left youtube for a month and THIS is what ha...,2020-08-11T16:34:06Z,UCYzPXprvl5Y-Sf0g4vX-m6g,jacksepticeye,24,2020-08-12T00:00:00Z,jacksepticeye|funny|funny meme|memes|jacksepti...,2038853,353787,2628,40221,https://i.ytimg.com/vi/J78aPJ3VyNs/default.jpg,False,False,I left youtube for a month and this is what ha...
3,kXLn3HkpjaA,XXL 2020 Freshman Class Revealed - Official An...,2020-08-11T16:38:55Z,UCbg_UMjlHJg_19SZckaKajg,XXL,10,2020-08-12T00:00:00Z,xxl freshman|xxl freshmen|2020 xxl freshman|20...,496771,23251,1856,7647,https://i.ytimg.com/vi/kXLn3HkpjaA/default.jpg,False,False,Subscribe to XXL → http://bit.ly/subscribe-xxl...
4,VIUo6yapDbc,Ultimate DIY Home Movie Theater for The LaBran...,2020-08-11T15:10:05Z,UCDVPcEbVLQgLZX0Rt6jo34A,Mr. Kate,26,2020-08-12T00:00:00Z,The LaBrant Family|DIY|Interior Design|Makeove...,1123889,45802,964,2196,https://i.ytimg.com/vi/VIUo6yapDbc/default.jpg,False,False,Transforming The LaBrant Family's empty white ...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
87586,FdzPV69i4Aw,Great Idea 💡 #shorts #tips #short,2021-10-12T12:00:12Z,UCGLDtG2tl0uG8P0eNstbLUA,Tool_Tips,26,2021-10-17T00:00:00Z,[None],3241407,27687,9037,937,https://i.ytimg.com/vi/FdzPV69i4Aw/default.jpg,False,False,NaN
87587,40_wdosHJaQ,I Was In The Funniest Minecraft Competition Wi...,2021-10-12T00:15:10Z,UCFbZ2e9IrPejOdp8wsKUxvA,QuackiTwo,24,2021-10-17T00:00:00Z,Quackity|Quackitwo|Quackity Second Channel|Qua...,425760,48882,215,1238,https://i.ytimg.com/vi/40_wdosHJaQ/default.jpg,False,False,STREAMED LIVE ON TWITCH: https://www.twitch.tv...
87588,cNG1QNQPlNE,Summoner Changes | FFXIV Endwalker Media Tour,2021-10-13T12:00:29Z,UCQjKMGUEzBmEHltb1OIMLUg,Larryzaur,20,2021-10-17T00:00:00Z,ffxiv|final fantasy xiv|ff14|final fantasy 14|...,203634,7065,52,754,https://i.ytimg.com/vi/cNG1QNQPlNE/default.jpg,False,False,Endwalker Media Tour Playlist ► https://www.yo...
87589,iYTK_1Sq8ZQ,Reaper Overview | FFXIV Endwalker Media Tour,2021-10-13T12:00:32Z,UCQjKMGUEzBmEHltb1OIMLUg,Larryzaur,20,2021-10-17T00:00:00Z,ffxiv|final fantasy xiv|ff14|final fantasy 14|...,210207,6884,43,540,https://i.ytimg.com/vi/iYTK_1Sq8ZQ/default.jpg,False,False,Endwalker Media Tour Playlist ► https://www.yo...


First, we shall take a simple look at categories and amount of videos trended from them.

If a category is not listed, then no videos have trended from it.

In [13]:
def print_dict_sort(arr, re=False, limit=-1):
    count = 0
    # code from: https://stackoverflow.com/questions/613183/how-do-i-sort-a-dictionary-by-value
    for i in sorted(arr.items(), key=lambda item: item[1], reverse=re):
        print(f'{i[1]}\t{i[0]}')
        if limit != -1:
            count += 1
            if count >= limit:
                break

In [14]:
category_count = {}
for i in data['categoryId']:
    temp = categories[f'{i}']
    try:
        category_count[temp] += 1
    except KeyError:
        category_count[temp] = 1

# code from: https://stackoverflow.com/questions/613183/how-do-i-sort-a-dictionary-by-value

print_dict_sort(category_count, True)

17615	Entertainment
16552	Music
14458	Gaming
9574	Sports
7524	People & Blogs
4721	Comedy
3364	Film & Animation
3315	News & Politics
3199	Science & Technology
2933	Howto & Style
1870	Education
1556	Autos & Vehicles
474	Pets & Animals
351	Travel & Events
85	Nonprofits & Activism


Below is functions used for extracting phrases in an array based on given k (max words in a phrase)

In [15]:
def phrasing(arr, k = 1):
    if len(arr) > k < 1:
        raise ValueError('Invalid array input or k value')
    ret = []
    for i in range(len(arr)+1-k):
        extract = arr[i:i+k]
        ret.append(' '.join(extract))
    return ret

def title_analysis(arr, w = 1):
    ret = {}
    for i in arr:
        t = i.lower()
        temp = phrasing(re.findall(r"\w+'*\w+", t), w)
        for k in temp:
            try:
                ret[k] += 1
            except KeyError:
                ret[k] = 1
    return ret

Commonly used words in trending video

In [41]:
print_dict_sort(title_analysis(data['title']), True, display_limit)

20190	the
11994	official
10694	video
8941	to
8132	in
5742	of
4805	vs
4796	and
4651	my
4555	on
4472	with
3931	music
3537	2021
3517	trailer
3507	you
3498	is
3482	highlights
3337	minecraft
3209	for
3038	new
2763	2020
2702	ft
2663	at
2377	game
2178	first
2164	shorts
2037	how
1945	from
1908	us
1821	this
1816	season
1815	lil
1733	it
1695	but
1636	feat
1621	we
1568	me
1554	live
1541	what
1513	among
1459	000
1386	day
1350	by
1313	full
1228	life
1227	out
1217	episode
1212	fortnite
1209	your
1184	audio
1183	world
1172	nba
1146	baby
1143	after
1140	why
1136	oficial
1129	10
1127	are
1111	best
1095	up
1088	every
1080	our
1080	challenge
1075	that
1055	night
1024	all
979	part
958	show
957	can
955	house
951	100
949	teaser
902	got
897	as
890	sports
886	no
886	bts
879	reveal
878	if
876	dream
854	home
843	time
841	update
827	not
825	reacts
824	an
816	one
814	about
794	ever
791	more
769	when
755	be
750	his
741	real
733	i'm
723	days
721	friday
712	most
693	off
693	nfl
693	week
692	final
690	review
688	over

Commonly used 2 word phrase in trending video

In [9]:
print_dict_sort(title_analysis(data['title'], 2), True, display_limit)

4432	official video
3558	music video
3328	official music
1497	among us
1258	official trailer
1103	video oficial
1002	in the
930	of the
921	official audio
800	to the
800	minecraft but
759	reacts to
709	how to
677	game highlights
668	friday night
632	on the
579	night funkin
548	from the
537	full game
525	at the
517	lyric video
492	for the
478	in minecraft
451	video ft
427	lil durk
424	in among
417	highlights nfl
413	the world
402	with the
400	cbs sports
399	lil baby
359	extended highlights
353	nfl 2020
345	the best
344	official teaser
341	the first
329	react to
319	bts 방탄소년단
319	jake paul
318	what happened
317	official lyric
314	behind the
305	the most
302	and the
298	justin bieber
297	this is
297	iphone 12
290	is the
286	24 hours
285	the new
284	the scenes
280	out of
273	nbc sports
270	apex legends
269	billie eilish
264	live from
262	first take
259	100 days
256	ft lil
253	ariana grande
248	highlights ucl
246	first time
245	dream smp
244	10 000
244	in fortnite
243	reveal trailer
241	lil 

Commonly used 3 word phrase in trending video

In [10]:
print_dict_sort(title_analysis(data['title'], 3), True, display_limit)

3328	official music video
579	friday night funkin
511	full game highlights
424	in among us
353	highlights nfl 2020
330	official video ft
312	official lyric video
284	behind the scenes
226	week highlights nfl
225	megan thee stallion
219	highlights ucl on
219	ucl on cbs
202	on cbs sports
198	extended highlights ucl
187	live from the
185	mod in among
183	try not to
179	america's got talent
177	call of duty
176	the first time
175	for 24 hours
173	for the first
167	premier league highlights
167	got talent 2021
164	in hardcore minecraft
162	wings hot ones
159	spicy wings hot
151	nba on tnt
149	the kid laroi
148	not to laugh
147	minecraft but you
145	smarter every day
145	in real life
142	never broke again
141	binging with babish
139	clash of clans
138	iphone 12 pro
137	among us but
136	to laugh challenge
136	official trailer hd
135	season official trailer
133	stephen reacts to
132	dead by daylight
130	cbs sports hq
128	machine gun kelly
128	youngboy never broke
127	remix official video
124	d

Commonly used 4 word phrase in trending video

In [11]:
print_dict_sort(title_analysis(data['title'], 4), True, display_limit)

219	highlights ucl on cbs
202	ucl on cbs sports
185	mod in among us
173	for the first time
169	extended highlights ucl on
167	america's got talent 2021
162	week highlights nfl 2020
159	spicy wings hot ones
148	try not to laugh
136	not to laugh challenge
128	youngboy never broke again
117	while eating spicy wings
117	eating spicy wings hot
117	role in among us
116	sneaker shopping with complex
109	black ops cold war
105	call of duty black
105	of duty black ops
105	duty black ops cold
103	goes sneaker shopping with
102	2021 bein sports usa
99	at lakers full game
99	lakers full game highlights
96	000 challenge zhc crafts
96	full game highlights june
95	lil baby lil durk
93	tiny desk home concert
92	episode breakdown ending explained
91	minecraft but you can
89	wins 000 challenge zhc
86	you suck at cooking
86	suck at cooking episode
85	official music video ft
84	to see if they
83	directed by cole bennett
83	spider man no way
83	man no way home
81	breakdown ending explained spoiler
81	endin

Process data relating to channels of trend videos

In [12]:
channel_id_tr = {}
channel_count = {}
display_count = {}
percent = [0, 0]

for _, i in data.iterrows():
    try:
        channel_count[i['channelId']] += 1
    except KeyError:
        channel_id_tr[i['channelId']] = i['channelTitle']
        channel_count[i['channelId']] = 1
        
for k, v in channel_count.items():
    display_count[channel_id_tr[k]] = v
    if v > 0: # just in case
        percent[int(v > 1)] += 1

del channel_id_tr
del channel_count

Channels and trend video count

In [13]:
print_dict_sort(display_count, True, display_limit)

550	NBA
528	NFL
458	SSSniperWolf
377	MrBeast Gaming
336	SSundee
316	MrBeast
310	ESPN
287	Champions League on CBS Sports
283	Saturday Night Live
280	Dude Perfect
267	NBC Sports
235	The Game Theorists
233	Fortnite
224	SpaceX
211	beIN SPORTS USA
210	Nintendo
204	The Try Guys
200	BLACKPINK
198	Big Hit Labels
198	Veritasium
197	The Food Theorists
196	SMTOWN
195	BANGTANTV
194	Marvel Entertainment
192	First We Feast
191	Apex Legends
191	ESPN FC
186	The Film Theorists
181	Warner Bros. Pictures
179	WWE
179	America's Got Talent
177	Skip and Shannon: UNDISPUTED
177	LazarBeam
176	MrBeast Shorts
172	Marques Brownlee
171	CalebCity
170	DAZN USA
169	JYP Entertainment
158	Dixie D'Amelio
157	Genshin Impact
156	Kurzgesagt – In a Nutshell
155	James Charles
154	Tom Scott
153	NikkieTutorials
153	ZHC Crafts
153	Matt Stonie
151	NBA on TNT
151	UFC - Ultimate Fighting Championship
149	Binging with Babish
149	Tucker Budzyn
149	FORMULA 1
148	Donut Media
146	jeffreestar
146	Drew Gooden
145	Simply Nailogical
145	Sm

In [14]:
print(f"{percent[1]} channels have multiple trend video and {percent[0]} have only one")

4216 channels have multiple trend video and 36 have only one
